# Classifying record pairs

There are dozens of classification algorithms for record linkage. In this example, several classification examples are showed. For these example, the [Krebs register](http://recordlinkage.readthedocs.org/en/latest/reference.html#recordlinkage.datasets.krebsregister_cmp_data) (German for cancer registry) dataset is used found in the [recordlinkage-datasets package](https://github.com/J535D165/recordlinkage-datasets). Be sure that this package is installed before trying the examples below. The Krebs register dataset contains comaprison vectors for which is known if the records belong to the same entity or not (so if they match or are distinct). This was done with a massive clerical review.

## Introduction
First, import the recordlinkage module and the Krebs register data. 

In [1]:
import recordlinkage
from recordlinkage.datasets import krebsregister_cmp_data

In [2]:
krebs_data, krebs_match = krebsregister_cmp_data(block=range(1,11))
krebs_data.head()

,,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
id1,id2,,,,,,,,,
49,6439,0.142857,NaN,0.000000,NaN,1,1.0,1.0,1.0,0.0
79667,83449,1.000000,NaN,0.250000,NaN,1,0.0,0.0,1.0,0.0
51539,59550,0.166667,NaN,0.100000,NaN,0,1.0,1.0,1.0,0.0
63018,66603,1.000000,NaN,0.000000,NaN,1,0.0,1.0,0.0,0.0
56779,79443,1.000000,NaN,0.166667,NaN,1,0.0,0.0,1.0,0.0


The dataset contains 5749132 compared record pairs and has attributes first name, last name, sex, birthday, birthmonth, birthyear and zipcode.

In [3]:
krebs_data.describe()

,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
count,5.748125e+06,103698.000000,5.749132e+06,2464.000000,5.749132e+06,5.748337e+06,5.748337e+06,5.748337e+06,5.736289e+06
mean,7.129025e-01,0.900018,3.156278e-01,0.318413,9.550014e-01,2.244653e-01,4.888553e-01,2.227486e-01,5.528661e-03
std,3.887584e-01,0.271318,3.342336e-01,0.368567,2.073011e-01,4.172297e-01,4.998758e-01,4.160910e-01,7.414915e-02
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.857143e-01,1.000000,1.000000e-01,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.000000,1.818182e-01,0.166667,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,1.000000,4.285714e-01,0.375000,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


The number of links in the data is:

In [4]:
len(krebs_match)

20931

Most of the classifiers in the ``recordlinkage`` package can not handle missing values in the comparison data. In the krebsregister dataset, two features are nearly always missing (cmp_firstname2 and cmp_lastname2). Also the other features are sometimes missing. To prevent issues with the classification algorithms, a widely used method is to declare missing values as disagreeing comparisons. So, use the power of ``pandas`` to replace the missing values quickly:

In [5]:
krebs_data.fillna(0, inplace=True)

## K-means classifier

The K-means clustering algorithm is well-known and widely used in big data analysis. The K-means classfier in the ``recordlinkage`` package is configured in such a way that it can be used for linking records. For more info about the K-means clustering see [wikipedia](https://en.wikipedia.org/wiki/K-means_clustering). 

In [6]:
kmeans = recordlinkage.KMeansClassifier()
result_kmeans = kmeans.learn(krebs_data)

type(result_kmeans)

pandas.indexes.multi.MultiIndex

The classifier is now trained and classified the comparison vectors. 

In [7]:
conf_kmeans = recordlinkage.confusion_matrix(krebs_match, result_kmeans, len(krebs_data))
conf_kmeans

array([[  20797,     134],
       [ 350725, 5377476]])

In [8]:
# The F-score for this classification is
recordlinkage.fscore(conf_kmeans)

0.10598466567971196

## Logistic regression

For this example, consider that the true match status of the first 5000 record pairs is known. 

In [9]:
golden_pairs = krebs_data[0:5000]
golden_matches_index = golden_pairs.index & krebs_match

The logistic classifier can be called in the same way as the k-means clustering algorithm. The only difference is now that the golden data is used for learning the algorithm. 

In [10]:
# Train the classifier
logreg = recordlinkage.LogisticRegressionClassifier()
logreg.learn(golden_pairs, golden_matches_index)

# Predict the match status for all record pairs
result_logreg = logreg.predict(krebs_data)

len(result_logreg)

20024

In [11]:
conf_logreg = recordlinkage.confusion_matrix(krebs_match, result_logreg, len(krebs_data))
conf_logreg

array([[  19831,    1100],
       [    193, 5728008]])

In [12]:
# The F-score for this classification is
recordlinkage.fscore(conf_logreg)

0.9684287632767672

## Support Vector Machines

Support vector machines have become increasingly popular in record linkage. See below why:

In [13]:
# Train the classifier
svm = recordlinkage.SVMClassifier()
svm.learn(golden_pairs, golden_matches_index)

# Predict the match status for all record pairs
result_svm = svm.predict(krebs_data)

len(result_svm)

20843

In [14]:
conf_svm = recordlinkage.confusion_matrix(krebs_match, result_svm, len(krebs_data))
conf_svm

array([[  20821,     110],
       [     22, 5728179]])

In [15]:
# The F-score for this classification is
recordlinkage.fscore(conf_svm)

0.9968401397998754

## Expectation/Conditional Maximization Algorithm

In [16]:
# Train the classifier
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((krebs_data > 0.8).astype(int))

len(result_ecm)

19817

In [17]:
conf_ecm = recordlinkage.confusion_matrix(krebs_match, result_ecm, len(krebs_data))
conf_ecm

array([[  19813,    1118],
       [      4, 5728197]])

In [18]:
# The F-score for this classification is
recordlinkage.fscore(conf_ecm)

0.9724649062530676